# Generate gene sets using several prompts.
Run this in the `llm2geneset` environment.

*Caution: This notebook uses the OpenAI API. Token costs can add up quickly.*

Generates JSON outputs in libs_human/{model}/*.json for downstream use in benchmarking.

In [1]:
import openai
from pathlib import Path
import json
import llm2geneset
import time
import pandas as pd

In [2]:
import re

def clean_elements(array):
    """Use regular expression to remove (GO:xxx) substring,  
       R-HSA-xxx substrings, and WPxxx substrings"""
    cleaned_array = []
    for element in array:
        cleaned_element = re.sub(r'\s*\(GO:\d+\)\s*|\s*R-HSA-\d+\s*|\s*WP\d+\s*', '', element)
        cleaned_array.append(cleaned_element)
    return cleaned_array

In [2]:
lib_names = ["KEGG_2021_Human", 
             "Reactome_2022", 
             "WikiPathway_2023_Human"]
aclient = openai.AsyncClient()
models = ["gpt-4o-mini-2024-07-18", "gpt-3.5-turbo-0125", "gpt-4o-2024-08-06"]

In [4]:
for model in models:
    for lib_name in lib_names:
        gmt = llm2geneset.read_gmt("libs_human/gmt/" + lib_name + ".txt")
        # Generate cleaned version of gene set description w/o identifiers.
        descr_cleaned = clean_elements(gmt["descr"])
        
        # Assemble and save generation results.
        gen_res = {}
        gen_res["lib_name"] = lib_name
        gen_res["model"] = model
        gen_res["descr"] = gmt["descr"]
        gen_res["descr_cleaned"] = descr_cleaned 
        gen_res["curated_genesets"] = gmt["genes"]
    
        with open('libs_human/' + model + '/' + lib_name + '.json', 'w') as json_file:
            json.dump(gen_res, json_file, indent=4)       


In [5]:
for model in models:
    print(model)
    for lib_name in lib_names:
        print(lib_name)
        
        with open("libs_human/" + model + "/" + lib_name + ".json") as f:
            gen_res = json.load(f)
        
        # Generate genes sets with a system message with role prompt.
        start_time1 = time.time()
        llm_genes_role = await llm2geneset.get_genes_bench(aclient,
                                                           gen_res["descr_cleaned"],
                                                           model=model,
                                                           prompt_type='basic',
                                                           use_sysmsg=True)
        end_time1 = time.time()
        gen_time_role = end_time1 - start_time1

        # Generate gene sets without role prompt.
        start_time2 = time.time()
        llm_genes_norole = await llm2geneset.get_genes_bench(aclient,
                                                             gen_res["descr_cleaned"],
                                                             model=model)
        end_time2 = time.time()
        gen_time_norole = end_time2 - start_time2

        # Generate gene sets with reasoning.
        start_time3 = time.time()
        llm_genes_reason = await llm2geneset.get_genes_bench(aclient,
                                                             gen_res["descr_cleaned"],
                                                             model=model, 
                                                             prompt_type='reason')
        end_time3 = time.time()
        gen_time_reasoning = end_time3 - start_time3

        # Generate gene sets with confidence.
        start_time4 = time.time()
        llm_genes_conf = await llm2geneset.get_genes_bench(aclient,
                                                           gen_res["descr_cleaned"],
                                                           model=model, 
                                                           prompt_type='conf')
        end_time4 = time.time()
        gen_time_conf = end_time4 - start_time4


        # Assemble and save generation results.
        gen_res["gen_time_role"] = gen_time_role
        gen_res["gen_time_norole"] = gen_time_norole
        gen_res["gen_time_reasoning"] = gen_time_reasoning
        gen_res["gen_time_conf"] = gen_time_conf
        gen_res["llm_genes_role"] = llm_genes_role
        gen_res["llm_genes_norole"] = llm_genes_norole
        gen_res["llm_genes_reason"] = llm_genes_reason
        gen_res["llm_genes_conf"] = llm_genes_conf
    
        with open('libs_human/' + model + '/' + lib_name + '.json', 'w') as json_file:
            json.dump(gen_res, json_file, indent=4)

gpt-4o-mini-2024-07-18
KEGG_2021_Human


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 320/320 [00:24<00:00, 12.93it/s]


Reactome_2022


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1818/1818 [13:21<00:00,  2.27it/s]


WikiPathway_2023_Human


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 320/320 [00:34<00:00,  9.21it/s]


Reactome_2022


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1818/1818 [06:49<00:00,  4.44it/s]


WikiPathway_2023_Human


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 801/801 [00:41<00:00, 19.15it/s]


In [3]:
# Generate gene sets for ensembling.
for model in models:
    print(model)
    for lib_name in lib_names:
        print(lib_name)
        with open("libs_human/" + model + "/" + lib_name + ".json") as f:
            gen_res = json.load(f)

        # Note seed is needed for ensembling to get different genes each time.
        seed = 732456
        for i in range(4):
            start_time = time.time()
            gen_res["llm_ensemble_" + str(i)] = await llm2geneset.get_genes_bench(aclient,
                                                                                  gen_res["descr_cleaned"],
                                                                                  model=model,
                                                                                  seed=seed+i)
            end_time = time.time()
            gen_res["gen_time_ensemble_" + str(i)] = end_time - start_time

        with open('libs_human/' + model + '/' + lib_name + '.json', 'w') as json_file:
            json.dump(gen_res, json_file, indent=4)

gpt-4o-mini-2024-07-18
KEGG_2021_Human


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 320/320 [00:19<00:00, 16.46it/s]


Reactome_2022


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1818/1818 [01:38<00:00, 18.39it/s]


WikiPathway_2023_Human


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 320/320 [00:30<00:00, 10.59it/s]


Reactome_2022


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1818/1818 [02:52<00:00, 10.53it/s]


WikiPathway_2023_Human


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 801/801 [00:53<00:00, 15.03it/s]
